#Notebook consist of:


*   basic understanding about train data
*   filling null values using interploation
*   merging 3 data frames (train,farm,weahther)
*   making subsets based on farmids 
*   converting each farmid to csv  
*   test data aswell



#Mounting drive to google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#importing Requried Libraries

In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import seaborn as sns

#Loading the requried files

In [ ]:
train_df=pd.read_csv('/content/drive/MyDrive/Capstone/train_data.csv')
farm_df=pd.read_csv('/content/drive/MyDrive/Capstone/farm_data-1646897931981.csv')
train_weather=pd.read_csv('/content/drive/MyDrive/Capstone/train_weather-1646897968670.csv')

In [ ]:
# checking the number of rows and columns in train_df
train_df.shape

(20216100, 4)

In [ ]:
farm_df

,farm_id,operations_commencing_year,num_processing_plants,farm_area,farming_company,deidentified_location
0,fid_110884,2008.0,NaN,690.455096,Obery Farms,location 7369
1,fid_90053,2004.0,NaN,252.696160,Obery Farms,location 7369
2,fid_17537,1991.0,NaN,499.446528,Obery Farms,location 7369
3,fid_110392,2002.0,NaN,2200.407555,Obery Farms,location 7369
4,fid_62402,1975.0,NaN,10833.140121,Obery Farms,location 7369
...,...,...,...,...,...,...
1444,fid_41664,1914.0,NaN,1822.663957,Sanderson Farms,location 5150
1445,fid_14529,NaN,NaN,399.297094,Obery Farms,location 5150
1446,fid_46454,1997.0,NaN,1046.552295,Sanderson Farms,location 5150
1447,fid_24103,2001.0,NaN,2766.186825,Dole Food Company,location 5150


In [ ]:
# farm_df['farm_id'].nunique()

In [ ]:
farm_df['farming_company'].value_counts()
farm_df['farming_company'].nunique()

16

In [ ]:
train_weather

,timestamp,deidentified_location,temp_obs,cloudiness,wind_direction,dew_temp,pressure_sea_level,precipitation,wind_speed
0,2016-01-01 00:00:00,location 7369,25.0,6.0,0.0,20.0,1019.7,NaN,0.0
1,2016-01-01 01:00:00,location 7369,24.4,NaN,70.0,21.1,1020.2,-1.0,1.5
2,2016-01-01 02:00:00,location 7369,22.8,2.0,0.0,21.1,1020.2,0.0,0.0
3,2016-01-01 03:00:00,location 7369,21.1,2.0,0.0,20.6,1020.1,0.0,0.0
4,2016-01-01 04:00:00,location 7369,20.0,2.0,250.0,20.0,1020.0,-1.0,2.6
...,...,...,...,...,...,...,...,...,...
139768,2016-12-31 19:00:00,location 5150,3.0,NaN,180.0,-8.0,NaN,NaN,5.7
139769,2016-12-31 20:00:00,location 5150,2.8,2.0,180.0,-8.9,1007.4,NaN,7.7
139770,2016-12-31 21:00:00,location 5150,2.8,NaN,180.0,-7.2,1007.5,NaN,5.1
139771,2016-12-31 22:00:00,location 5150,2.2,NaN,170.0,-6.7,1008.0,NaN,4.6


#checking null values

In [ ]:
#train data
train_df.isnull().sum()

date               0
farm_id            0
ingredient_type    0
yield              0
dtype: int64

In [ ]:
# farm data
farm_df.isnull().sum()*100/len(farm_df)

farm_id                        0.000000
operations_commencing_year    53.416149
num_processing_plants         75.500345
farm_area                      0.000000
farming_company                0.000000
deidentified_location          0.000000
dtype: float64

In [ ]:
# farm data
train_weather.isnull().sum()*100/len(train_weather)

timestamp                 0.000000
deidentified_location     0.000000
temp_obs                  0.039350
cloudiness               49.489529
wind_direction            4.484414
dew_temp                  0.080845
pressure_sea_level        7.596603
precipitation            35.979052
wind_speed                0.217496
dtype: float64

# Droping unwanted attributes

In [ ]:
# droping the columns which have more then 40% null values
train_weather.drop(['cloudiness'],axis=1,inplace=True)
farm_df.drop(['operations_commencing_year','num_processing_plants'],axis=1,inplace=True)

In [ ]:
# taking the null columns into a list
null_atr=['temp_obs','wind_direction','dew_temp','pressure_sea_level','precipitation','wind_speed']

In [ ]:
# creating funtion for filling the missing data
def interpolate_columns(df, columns):
    for col in columns:
        df[col] = df[col].interpolate()
    return df

In [ ]:
train_weather=interpolate_columns(train_weather,null_atr)

In [ ]:
train_weather.isnull().sum()

timestamp                0
deidentified_location    0
temp_obs                 0
wind_direction           0
dew_temp                 0
pressure_sea_level       0
precipitation            1
wind_speed               0
dtype: int64

In [ ]:
train_weather = train_weather.dropna(axis=0)

#merging train data and farm data

In [ ]:
train_data1=pd.merge(train_df,farm_df,on='farm_id')

In [ ]:
train_data1

,date,farm_id,ingredient_type,yield,farm_area,farming_company,deidentified_location
0,2016-01-01 00:00:00,fid_110884,ing_w,0.00,690.455096,Obery Farms,location 7369
1,2016-01-01 01:00:00,fid_110884,ing_w,0.00,690.455096,Obery Farms,location 7369
2,2016-01-01 02:00:00,fid_110884,ing_w,0.00,690.455096,Obery Farms,location 7369
3,2016-01-01 03:00:00,fid_110884,ing_w,0.00,690.455096,Obery Farms,location 7369
4,2016-01-01 04:00:00,fid_110884,ing_w,0.00,690.455096,Obery Farms,location 7369
...,...,...,...,...,...,...,...
20602894,2016-12-31 19:00:00,fid_19864,ing_w,43.66,4598.698500,Obery Farms,location 5290
20602895,2016-12-31 20:00:00,fid_19864,ing_w,43.64,4598.698500,Obery Farms,location 5290
20602896,2016-12-31 21:00:00,fid_19864,ing_w,43.89,4598.698500,Obery Farms,location 5290
20602897,2016-12-31 22:00:00,fid_19864,ing_w,44.37,4598.698500,Obery Farms,location 5290


In [ ]:
# converting  column name timestamp  to date name in weather data ,used for merging
train_weather.rename(columns = {'timestamp':'date'}, inplace = True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


train_data1,train_weather merging

In [ ]:
main_data=pd.merge(train_data1,train_weather,on=['deidentified_location','date'],)#.sort('date')

In [ ]:
main_data.to_csv('/content/drive/MyDrive/Capstone/merged data/merged_train_data.csv',index=False)

In [ ]:
main_data

,date,farm_id,ingredient_type,yield,farm_area,farming_company,deidentified_location,temp_obs,wind_direction,dew_temp,pressure_sea_level,precipitation,wind_speed
0,2016-01-01 01:00:00,fid_110884,ing_w,0.000,690.455096,Obery Farms,location 7369,24.4,70.0,21.1,1020.2,-1.0,1.5
1,2016-01-01 01:00:00,fid_90053,ing_w,0.000,252.696160,Obery Farms,location 7369,24.4,70.0,21.1,1020.2,-1.0,1.5
2,2016-01-01 01:00:00,fid_17537,ing_w,0.000,499.446528,Obery Farms,location 7369,24.4,70.0,21.1,1020.2,-1.0,1.5
3,2016-01-01 01:00:00,fid_110392,ing_w,0.000,2200.407555,Obery Farms,location 7369,24.4,70.0,21.1,1020.2,-1.0,1.5
4,2016-01-01 01:00:00,fid_62402,ing_w,0.000,10833.140121,Obery Farms,location 7369,24.4,70.0,21.1,1020.2,-1.0,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20511423,2016-12-17 06:00:00,fid_117010,ing_x,0.000,11857.395696,Obery Farms,location 868,-17.1,60.0,-19.2,1022.7,2.0,2.6
20511424,2016-12-17 06:00:00,fid_117010,ing_z,246.516,11857.395696,Obery Farms,location 868,-17.1,60.0,-19.2,1022.7,2.0,2.6
20511425,2016-12-17 06:00:00,fid_74945,ing_w,92.000,7561.375170,Obery Farms,location 868,-17.1,60.0,-19.2,1022.7,2.0,2.6
20511426,2016-12-17 06:00:00,fid_74945,ing_x,0.000,7561.375170,Obery Farms,location 868,-17.1,60.0,-19.2,1022.7,2.0,2.6


In [ ]:
data_w=main_data[main_data['ingredient_type']=='ing_w']
data_w['ingredient_type'].nunique()

1

In [ ]:
data_w.to_csv('/content/drive/MyDrive/Capstone/ingredient type/ing_sort/data_w',index=False)

In [ ]:
# train_df[train_df['farm_id']=='fid_110884']

In [ ]:
# unique_ids =train_df['farm_id'].unique()

In [ ]:
# unique_ids1=['fid_110884','fid_90053','fid_17537']

In [ ]:
# #subsets={}
# for i in unique_ids1:
#   subset=train_df[train_df['farm_id']==i]
#   subsets[i] = subset
#   # subsets.append(subset)


In [ ]:
#taking copy of main_data into train_data 
train_data=main_data.copy()

In [ ]:
train_data.isnull().sum()*100/len(train_data)

date                     0.0
farm_id                  0.0
ingredient_type          0.0
yield                    0.0
farm_area                0.0
farming_company          0.0
deidentified_location    0.0
temp_obs                 0.0
wind_direction           0.0
dew_temp                 0.0
pressure_sea_level       0.0
precipitation            0.0
wind_speed               0.0
dtype: float64

In [ ]:
print('minimum date',train_data['date'].min())
print('maximum date',train_data['date'].max())


minimum date 2016-01-01 00:00:00
maximum date 2016-12-31 23:00:00


In [ ]:
# train_data[train_data['date']=='2016-01-01 00:00:00']

In [ ]:
# train_df[train_df['farm_id']=='fid_110884']

In [ ]:
train_data.set_index('date',inplace=True)


In [ ]:
#head of train_data ,displayes  top 5 rows 
train_data.head()

,farm_id,ingredient_type,yield,farm_area,farming_company,deidentified_location,temp_obs,wind_direction,dew_temp,pressure_sea_level,precipitation,wind_speed
date,,,,,,,,,,,,
2016-01-01 01:00:00,fid_110884,ing_w,0.0,690.455096,Obery Farms,location 7369,24.4,70.0,21.1,1020.2,-1.0,1.5
2016-01-01 01:00:00,fid_90053,ing_w,0.0,252.696160,Obery Farms,location 7369,24.4,70.0,21.1,1020.2,-1.0,1.5
2016-01-01 01:00:00,fid_17537,ing_w,0.0,499.446528,Obery Farms,location 7369,24.4,70.0,21.1,1020.2,-1.0,1.5
2016-01-01 01:00:00,fid_110392,ing_w,0.0,2200.407555,Obery Farms,location 7369,24.4,70.0,21.1,1020.2,-1.0,1.5
2016-01-01 01:00:00,fid_62402,ing_w,0.0,10833.140121,Obery Farms,location 7369,24.4,70.0,21.1,1020.2,-1.0,1.5


In [ ]:
train_data.sort_index(axis=0)


,farm_id,ingredient_type,yield,farm_area,farming_company,deidentified_location,temp_obs,wind_direction,dew_temp,pressure_sea_level,precipitation,wind_speed
date,,,,,,,,,,,,
2016-01-01 00:00:00,fid_17087,ing_w,3.8750,33.073468,Sanderson Farms,location 5677,25.0,0.0,20.0,1019.7,2.000000,0.0
2016-01-01 00:00:00,fid_88128,ing_y,1539.0600,14636.403135,Wayne Farms,location 2532,-8.3,250.0,-12.2,1023.3,-0.999315,4.1
2016-01-01 00:00:00,fid_62756,ing_w,28.8980,12239.505735,Wayne Farms,location 2532,-8.3,250.0,-12.2,1023.3,-0.999315,4.1
2016-01-01 00:00:00,fid_62756,ing_x,0.0000,12239.505735,Wayne Farms,location 2532,-8.3,250.0,-12.2,1023.3,-0.999315,4.1
2016-01-01 00:00:00,fid_62756,ing_y,812.5000,12239.505735,Wayne Farms,location 2532,-8.3,250.0,-12.2,1023.3,-0.999315,4.1
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-31 23:00:00,fid_70492,ing_w,30.2700,1047.388422,Sanderson Farms,location 5290,8.9,200.0,-5.6,1015.5,0.000000,6.2
2016-12-31 23:00:00,fid_35085,ing_w,85.7800,25204.583900,Obery Farms,location 5290,8.9,200.0,-5.6,1015.5,0.000000,6.2
2016-12-31 23:00:00,fid_109740,ing_w,113.5000,8000.806360,Sanderson Farms,location 5290,8.9,200.0,-5.6,1015.5,0.000000,6.2


In [ ]:
train_data['deidentified_location'].value_counts()

location 2532    2790470
location 8421    2703006
location 5489    2580537
location 5410    2525792
location 5290    2439148
location 5150    1760872
location 7369    1085342
location 6364     819743
location 5833     799257
location 565      667989
location 5677     575401
location 959      552034
location 1784     411313
location 4525     359642
location 7048     323623
location 868      117259
Name: deidentified_location, dtype: int64

In [ ]:
# unique_ids1=['fid_110884','fid_90053','fid_17537']

#individual farm_ids

In [ ]:
# from entire train data taking unique farm ids
unique_ids =train_data['farm_id'].unique()
# 10 farm ids
ids=unique_ids[:10]

# creating subsets of train data based on farm ids

In [ ]:
subsets={}
for i in ids:
  subset=train_data[train_data['farm_id']==i]
  subsets[i] = subset

In [ ]:
subsets

{'fid_110884':                         farm_id ingredient_type    yield   farm_area  \
 date                                                                   
 2016-01-01 01:00:00  fid_110884           ing_w    0.000  690.455096   
 2016-01-01 02:00:00  fid_110884           ing_w    0.000  690.455096   
 2016-01-01 03:00:00  fid_110884           ing_w    0.000  690.455096   
 2016-01-01 04:00:00  fid_110884           ing_w    0.000  690.455096   
 2016-01-01 05:00:00  fid_110884           ing_w    0.000  690.455096   
 ...                         ...             ...      ...         ...   
 2016-12-31 19:00:00  fid_110884           ing_w  232.071  690.455096   
 2016-12-31 20:00:00  fid_110884           ing_w  189.069  690.455096   
 2016-12-31 21:00:00  fid_110884           ing_w  169.958  690.455096   
 2016-12-31 22:00:00  fid_110884           ing_w  169.958  690.455096   
 2016-12-31 23:00:00  fid_110884           ing_w  175.418  690.455096   
 
                     farming_compan

In [ ]:
path='/content/drive/MyDrive/Capstone/all ingredient subsets'

#converting each farm_id subset into csv file

In [ ]:
%%time
for i in subsets:   #The for loop iterates over each element i in the sequence subsets.
  subsets[i].to_csv(path+'/'+i+'.csv')  #The / symbol is used to separate the directory and the file name,
                                  # and the + symbol is used to concatenate the strings.

CPU times: user 873 ms, sys: 43.6 ms, total: 917 ms
Wall time: 5.61 s


In [ ]:
import glob
# used to search for all files in the directory specified by path 

# reading all the files at a time using glob

In [ ]:
# create empty list
list_of_names=[] 
#The glob.glob function is used to search for all files in the directory specified by path 
#that match the pattern *.csv
for i in glob.glob(path+'/'+"*.csv"):
  list_of_names.append(i)  

In [ ]:
 #create empty list
dataframes_list = []
#For each iteration of the loop, the line temp_df=pd.read_csv(list_of_names[i]) reads the .csv file 
#at the index i of the list list_of_names into a pandas dataframe 
for i in range (len(list_of_names)):
  temp_df=pd.read_csv(list_of_names[i])
  dataframes_list.append(temp_df)

In [ ]:
for i in dataframes_list:
  print(i.shape)

(8783, 13)
(8783, 13)
(8783, 13)
(8783, 13)
(8783, 13)
(8783, 13)
(8783, 13)
(16050, 13)
(8783, 13)
(16048, 13)


#Test Data

In [ ]:
test_df=pd.read_csv('/content/drive/MyDrive/Capstone/test_data-1664552867678.csv')
test_weather=pd.read_csv('/content/drive/MyDrive/Capstone/test_weather-1646897984996-1664552604982.csv')
farm_df=pd.read_csv('/content/drive/MyDrive/Capstone/farm_data-1646897931981.csv')

In [ ]:
test_df

,date,farm_id,ingredient_type,id
0,2017-01-01 00:00:00,fid_110884,ing_w,0
1,2017-01-01 00:00:00,fid_90053,ing_w,1
2,2017-01-01 00:00:00,fid_17537,ing_w,2
3,2017-01-01 00:00:00,fid_110392,ing_w,3
4,2017-01-01 00:00:00,fid_62402,ing_w,4
...,...,...,...,...
20848795,2017-12-25 07:00:00,fid_58769,ing_x,41696845
20848796,2017-12-25 07:00:00,fid_58769,ing_z,41696846
20848797,2017-12-25 07:00:00,fid_117010,ing_w,41696847
20848798,2017-12-25 07:00:00,fid_117010,ing_x,41696848


In [ ]:
test_df.shape

(20848800, 4)

In [ ]:
test_df.isnull().sum()

date               0
farm_id            0
ingredient_type    0
id                 0
dtype: int64

In [ ]:
test_weather

,timestamp,deidentified_location,temp_obs,cloudiness,wind_direction,dew_temp,pressure_sea_level,precipitation,wind_speed
0,2017-01-01 00:00:00,location 7369,17.8,4.0,100.0,11.7,1021.4,NaN,3.6
1,2017-01-01 01:00:00,location 7369,17.8,2.0,130.0,12.8,1022.0,0.0,3.1
2,2017-01-01 02:00:00,location 7369,16.1,0.0,140.0,12.8,1021.9,0.0,3.1
3,2017-01-01 03:00:00,location 7369,17.2,0.0,140.0,13.3,1022.2,0.0,3.1
4,2017-01-01 04:00:00,location 7369,16.7,2.0,130.0,13.3,1022.3,0.0,2.6
...,...,...,...,...,...,...,...,...,...
138820,2017-12-31 19:00:00,location 5150,-15.0,NaN,310.0,-18.3,1027.5,-1.0,5.1
138821,2017-12-31 20:00:00,location 5150,-15.0,NaN,310.0,-18.9,1027.6,-1.0,6.2
138822,2017-12-31 21:00:00,location 5150,-15.0,NaN,290.0,-19.4,1028.0,-1.0,6.7
138823,2017-12-31 22:00:00,location 5150,-15.6,NaN,290.0,-19.4,1028.4,NaN,6.7


In [ ]:
test_weather.shape

(138825, 9)

In [ ]:
test_weather.isnull().sum()

timestamp                    0
deidentified_location        0
temp_obs                    43
cloudiness               69065
wind_direction            6134
dew_temp                   184
pressure_sea_level       10883
precipitation            47788
wind_speed                 240
dtype: int64

In [ ]:
farm_df

,farm_id,operations_commencing_year,num_processing_plants,farm_area,farming_company,deidentified_location
0,fid_110884,2008.0,NaN,690.455096,Obery Farms,location 7369
1,fid_90053,2004.0,NaN,252.696160,Obery Farms,location 7369
2,fid_17537,1991.0,NaN,499.446528,Obery Farms,location 7369
3,fid_110392,2002.0,NaN,2200.407555,Obery Farms,location 7369
4,fid_62402,1975.0,NaN,10833.140121,Obery Farms,location 7369
...,...,...,...,...,...,...
1444,fid_41664,1914.0,NaN,1822.663957,Sanderson Farms,location 5150
1445,fid_14529,NaN,NaN,399.297094,Obery Farms,location 5150
1446,fid_46454,1997.0,NaN,1046.552295,Sanderson Farms,location 5150
1447,fid_24103,2001.0,NaN,2766.186825,Dole Food Company,location 5150


In [ ]:
test_data2=pd.merge(test_df,farm_df,on='farm_id',how='inner')
test_data2

,date,farm_id,ingredient_type,id,operations_commencing_year,num_processing_plants,farm_area,farming_company,deidentified_location
0,2017-01-01 00:00:00,fid_110884,ing_w,0,2008.0,NaN,690.455096,Obery Farms,location 7369
1,2017-01-01 01:00:00,fid_110884,ing_w,129,2008.0,NaN,690.455096,Obery Farms,location 7369
2,2017-01-01 02:00:00,fid_110884,ing_w,258,2008.0,NaN,690.455096,Obery Farms,location 7369
3,2017-01-01 03:00:00,fid_110884,ing_w,387,2008.0,NaN,690.455096,Obery Farms,location 7369
4,2017-01-01 04:00:00,fid_110884,ing_w,516,2008.0,NaN,690.455096,Obery Farms,location 7369
...,...,...,...,...,...,...,...,...,...
21251755,2017-12-28 07:00:00,fid_34139,ing_w,41673195,2001.0,NaN,8572.252713,Wayne Farms,location 5150
21251756,2017-12-28 08:00:00,fid_34139,ing_w,41673445,2001.0,NaN,8572.252713,Wayne Farms,location 5150
21251757,2017-12-28 09:00:00,fid_34139,ing_w,41673695,2001.0,NaN,8572.252713,Wayne Farms,location 5150
21251758,2017-12-28 10:00:00,fid_34139,ing_w,41673945,2001.0,NaN,8572.252713,Wayne Farms,location 5150


In [ ]:
test_data2.isnull().sum()

date                                 0
farm_id                              0
ingredient_type                      0
id                                   0
operations_commencing_year    12526800
num_processing_plants         17520000
farm_area                            0
farming_company                      0
deidentified_location                0
dtype: int64

In [ ]:
# dropping the columns which as more then 40% null values from test_weather and est_data2
test_weather.drop(['cloudiness'],axis=1,inplace=True)
test_data2.drop(['operations_commencing_year','num_processing_plants'],axis=1,inplace=True)

In [ ]:
test_weather['precipitation'].isnull()

0          True
1         False
2         False
3         False
4         False
          ...  
138820    False
138821    False
138822    False
138823     True
138824     True
Name: precipitation, Length: 138825, dtype: bool

In [ ]:
test_weather.rename(columns={'timestamp':'date'},inplace=True)

In [ ]:
test_weather.columns

Index(['date', 'deidentified_location', 'temp_obs', 'wind_direction',
       'dew_temp', 'pressure_sea_level', 'precipitation', 'wind_speed'],
      dtype='object')

In [ ]:
test_data=pd.merge(test_data2,test_weather,on=['date','deidentified_location'],how='inner')

In [ ]:
test_data.isnull().sum()

date                           0
farm_id                        0
ingredient_type                0
id                             0
farm_area                      0
farming_company                0
deidentified_location          0
temp_obs                   11052
wind_direction           1384647
dew_temp                   41457
pressure_sea_level       1245052
precipitation            3832708
wind_speed                 56803
dtype: int64

In [ ]:
columns_to_interpolate = ['temp_obs', 'wind_direction', 'dew_temp', 'pressure_sea_level', 'precipitation', 'wind_speed']
test_data = interpolate_columns(test_data, columns_to_interpolate)

In [ ]:
test_data.isnull().sum()

date                       0
farm_id                    0
ingredient_type            0
id                         0
farm_area                  0
farming_company            0
deidentified_location      0
temp_obs                   0
wind_direction             0
dew_temp                   0
pressure_sea_level         0
precipitation            130
wind_speed                 0
dtype: int64

In [ ]:
test_data.fillna(0,inplace=True)

In [ ]:
test_data.isnull().sum()

date                     0
farm_id                  0
ingredient_type          0
id                       0
farm_area                0
farming_company          0
deidentified_location    0
temp_obs                 0
wind_direction           0
dew_temp                 0
pressure_sea_level       0
precipitation            0
wind_speed               0
dtype: int64

In [ ]:
test_w=test_data[test_data['ingredient_type']=='ing_w']

In [ ]:
test_w

,date,farm_id,ingredient_type,id,farm_area,farming_company,deidentified_location,temp_obs,wind_direction,dew_temp,pressure_sea_level,precipitation,wind_speed
0,2017-01-01 00:00:00,fid_110884,ing_w,0,690.455096,Obery Farms,location 7369,17.8,100.0,11.7,1021.4,0.0,3.6
1,2017-01-01 00:00:00,fid_90053,ing_w,1,252.696160,Obery Farms,location 7369,17.8,100.0,11.7,1021.4,0.0,3.6
2,2017-01-01 00:00:00,fid_17537,ing_w,2,499.446528,Obery Farms,location 7369,17.8,100.0,11.7,1021.4,0.0,3.6
3,2017-01-01 00:00:00,fid_110392,ing_w,3,2200.407555,Obery Farms,location 7369,17.8,100.0,11.7,1021.4,0.0,3.6
4,2017-01-01 00:00:00,fid_62402,ing_w,4,10833.140121,Obery Farms,location 7369,17.8,100.0,11.7,1021.4,0.0,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21154783,2017-12-31 23:00:00,fid_74945,ing_w,26001706,7561.375170,Obery Farms,location 868,-21.1,270.0,-26.8,1029.8,2.0,4.6
21154786,2017-12-31 23:00:00,fid_36739,ing_w,26001709,14173.188777,Obery Farms,location 868,-21.1,270.0,-26.8,1029.8,2.0,4.6
21154788,2017-12-31 23:00:00,fid_112848,ing_w,26001711,6320.191090,Obery Farms,location 868,-21.1,270.0,-26.8,1029.8,2.0,4.6
21154791,2017-12-31 23:00:00,fid_58769,ing_w,26001714,8659.117018,Obery Farms,location 868,-21.1,270.0,-26.8,1029.8,2.0,4.6


In [ ]:
test_w['ingredient_type'].nunique()

1

In [ ]:
test_w.to_csv('/content/drive/MyDrive/Capstone/ingredient type/test_w',index=False)